In [2]:
#from detectron2.config import get_cfg
import os
#import torch
import numpy as np
import cv2
import random
import json

#from detectron2.engine import DefaultPredictor
#from detectron2.evaluation import inference_on_dataset, COCOEvaluator
#from detectron2.modeling import build_model

#from detectron2.checkpoint import DetectionCheckpointer

#import detectron2.data.transforms as T
#from detectron2.data import DatasetMapper   # the default mapper
#from detectron2.data import build_detection_train_loader, build_detection_test_loader

from detectron2.data.datasets import register_coco_instances
#from detectron2 import model_zoo

#import DatasetCatalog, MetadataCatalog
from detectron2.data import DatasetCatalog, MetadataCatalog

#import logger
from detectron2.utils.logger import setup_logger
setup_logger()

from utils.structure_utils import map1_cr, map2_cr, structure1_cr, structure2_cr, single_syn_cr

# Import custom dataset mapper for augmentation
#from data_augmentation.augmentation_datasetmapper import AugmentDatasetMapper
#from data_augmentation.augmentations import create_augmentations
#from data_augmentation.augmentations_utils import create_bbox_hflip_indices, create_bbox_vflip_indices

#import visualizer
#from detectron2.utils.visualizer import Visualizer

import sahi
from sahi import AutoDetectionModel
from sahi.predict import predict, get_prediction, get_sliced_prediction
from sahi.scripts.coco_evaluation import evaluate
from sahi.scripts.coco_error_analysis import analyse



* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


c:\Users\wouds\Documents\GitHub\Flowsheet_Digitization


In [3]:
train_dataset_annot_path = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Data\SWMI\SWMI_baseline\test.json"

# alternative way to get the categories from the json
with open(train_dataset_annot_path) as f:
    train_dataset_annot = json.load(f)

# get the categories from the json
categories = train_dataset_annot['categories']

# set up category mapping
category_mapping = {}
for category in categories:
    category_mapping[str(category['id'])] = category['name']

print(category_mapping)

{'1': 'Agglomerator', '2': 'Agitator', '3': 'AgitatorAnchor', '4': 'AgitatorCrossBeam', '5': 'AgitatorDisc', '6': 'AgitatorFlatBladePaddle', '7': 'AgitatorGatPaddle', '8': 'AgitatorGeneric', '9': 'AgitatorHelical', '10': 'AgitatorImpeller', '11': 'AgitatorPropeller', '12': 'AgitatorTurbine', '13': 'AirCoolingSystem', '14': 'AirEjector', '15': 'AlternateChoiceOrSafetyInstrumentedSystem', '16': 'AlternateChoiceOrSafetyInstrumentedSystemField', '17': 'AlternateChoiceOrSafetyInstrumentedSystemFrontCentralPanel', '18': 'AlternateChoiceOrSafetyInstrumentedSystemFrontSecondaryPanel', '19': 'AlternateChoiceOrSafetyInstrumentedSystemRearCentralPanel', '20': 'AlternateChoiceOrSafetyInstrumentedSystemRearSecondaryPanel', '21': 'AlternatingCurrentGenerator', '22': 'AlternatingCurrentMotor', '23': 'AnalysisPrimaryElement', '24': 'AngleBallValve', '25': 'AngleGlobeValve', '26': 'AnglePlugValve', '27': 'AngleValve', '28': 'AngleValveBlowdown', '29': 'AngleValveGeneric', '30': 'AngleValveHandOperated'

In [4]:
#model_path = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Experiments\8. Exp_SWMI_baseline_2\results\output\model_final.pth"
config_path = "COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"

#train_dataset_img_path = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Data\SWMI\images"
#train_dataset_annot_path = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Data\SWMI\SWMI_baseline\train.json"

#args
#model_path = r"C:\Users\wouds\Documents\GitHub\Flowsheet_Digitization\experiments\output\ASPEN_1\output\model_final.pth"
model_path = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Experiments\11. Exp_SWMI_baseline_seg_512_val_seg\results\output\model_0074999.pth"
config_path = "COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"

#train_dataset_annot_path = r"C:\Users\wouds\Documents\GitHub\Flowsheet_Digitization\data\ASPEN_1\train.json"
train_dataset_annot_path = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Data\SWMI\SWMI_baseline\test.json"
#train_dataset_img_path = r"C:\Users\wouds\Documents\GitHub\Flowsheet_Digitization\data\ASPEN_1\images"
train_dataset_img_path = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Data\SWMI\images"

try:
    register_coco_instances(name='train', metadata={}, json_file=train_dataset_annot_path, image_root=train_dataset_img_path)
except:
    DatasetCatalog.remove('train')
    MetadataCatalog.remove('train')
    register_coco_instances(name='train', metadata={}, json_file=train_dataset_annot_path, image_root=train_dataset_img_path)
dataset = DatasetCatalog.get('train')
metadata = MetadataCatalog.get('train')
category_names = metadata.thing_classes
print(category_names)
category_mapping = {
                        str(ind): category_name for ind, category_name in enumerate(category_names)
                    }

model = AutoDetectionModel.from_pretrained(model_type='detectron2',
    model_path=model_path,
    config_path=config_path,
    confidence_threshold=0.5,
    device="cuda", # or 'cuda:0'
    category_mapping=category_mapping, 
)

print(category_mapping)

[12/12 17:01:27 d2.data.datasets.coco]: Loaded 89 images in COCO format from W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Data\SWMI\SWMI_baseline\test.json


['Agglomerator', 'Agitator', 'AgitatorAnchor', 'AgitatorCrossBeam', 'AgitatorDisc', 'AgitatorFlatBladePaddle', 'AgitatorGatPaddle', 'AgitatorGeneric', 'AgitatorHelical', 'AgitatorImpeller', 'AgitatorPropeller', 'AgitatorTurbine', 'AirCoolingSystem', 'AirEjector', 'AlternateChoiceOrSafetyInstrumentedSystem', 'AlternateChoiceOrSafetyInstrumentedSystemField', 'AlternateChoiceOrSafetyInstrumentedSystemFrontCentralPanel', 'AlternateChoiceOrSafetyInstrumentedSystemFrontSecondaryPanel', 'AlternateChoiceOrSafetyInstrumentedSystemRearCentralPanel', 'AlternateChoiceOrSafetyInstrumentedSystemRearSecondaryPanel', 'AlternatingCurrentGenerator', 'AlternatingCurrentMotor', 'AnalysisPrimaryElement', 'AngleBallValve', 'AngleGlobeValve', 'AnglePlugValve', 'AngleValve', 'AngleValveBlowdown', 'AngleValveGeneric', 'AngleValveHandOperated', 'AngleValveMotor', 'AngleValvePneumaticDiaphragm', 'AveragingPitotTube', 'AxialBlower', 'AxialCompressor', 'AxialFan', 'BackPressureRegulator', 'BackPressureRegulatorExt

12/12/2023 17:01:29 - INFO - fvcore.common.checkpoint -   [Checkpointer] Loading from w:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Experiments\11. Exp_SWMI_baseline_seg_512_val_seg\results\output\model_0074999.pth ...


{'0': 'Agglomerator', '1': 'Agitator', '2': 'AgitatorAnchor', '3': 'AgitatorCrossBeam', '4': 'AgitatorDisc', '5': 'AgitatorFlatBladePaddle', '6': 'AgitatorGatPaddle', '7': 'AgitatorGeneric', '8': 'AgitatorHelical', '9': 'AgitatorImpeller', '10': 'AgitatorPropeller', '11': 'AgitatorTurbine', '12': 'AirCoolingSystem', '13': 'AirEjector', '14': 'AlternateChoiceOrSafetyInstrumentedSystem', '15': 'AlternateChoiceOrSafetyInstrumentedSystemField', '16': 'AlternateChoiceOrSafetyInstrumentedSystemFrontCentralPanel', '17': 'AlternateChoiceOrSafetyInstrumentedSystemFrontSecondaryPanel', '18': 'AlternateChoiceOrSafetyInstrumentedSystemRearCentralPanel', '19': 'AlternateChoiceOrSafetyInstrumentedSystemRearSecondaryPanel', '20': 'AlternatingCurrentGenerator', '21': 'AlternatingCurrentMotor', '22': 'AnalysisPrimaryElement', '23': 'AngleBallValve', '24': 'AngleGlobeValve', '25': 'AnglePlugValve', '26': 'AngleValve', '27': 'AngleValveBlowdown', '28': 'AngleValveGeneric', '29': 'AngleValveHandOperated',

In [1]:
import yaml
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
import sahi
from sahi.predict import predict, get_prediction, get_sliced_prediction
from sahi import AutoDetectionModel

# config path
config_path = "COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"

# model path
model_path = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Experiments\11. Exp_SWMI_baseline_seg_512_val_seg\results\output\model_0074999.pth"

# image path
image_path = r"C:\Users\wouds\Documents\GitHub\Flowsheet_Digitization\data\ASPEN_1\images\images\Aspen PFD_p11-1766.png"

with open("config/config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

dataset_train_annos = config['dataset']['train']['annot_file_path']
dataset_train_images = config['dataset']['train']['images_path']


try:
    register_coco_instances(name='train', metadata={}, json_file=dataset_train_annos, image_root=dataset_train_images)
except:
    DatasetCatalog.remove('train')
    MetadataCatalog.remove('train')
    register_coco_instances(name='train', metadata={}, json_file=dataset_train_annos, image_root=dataset_train_images)
dataset = DatasetCatalog.get('train')
metadata = MetadataCatalog.get('train')
category_names = metadata.thing_classes

# category_mapping
category_mapping = {
                        str(ind): category_name for ind, category_name in enumerate(category_names)
                    }

model = AutoDetectionModel.from_pretrained(model_type='detectron2',
    model_path=model_path,
    config_path=config,
    confidence_threshold=0.5,
    device="cuda", # or 'cuda:0'
    category_mapping=category_mapping, 
)


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


c:\Users\wouds\Documents\GitHub\Flowsheet_Digitization
[12/12 15:48:50 d2.data.datasets.coco]: Loaded 80 images in COCO format from ./data/ASPEN_1/train.json
[12/12 15:48:51 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Experiments\11. Exp_SWMI_baseline_seg_512_val_seg\results\output\model_0074999.pth ...


12/12/2023 15:48:51 - INFO - fvcore.common.checkpoint -   [Checkpointer] Loading from w:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Experiments\11. Exp_SWMI_baseline_seg_512_val_seg\results\output\model_0074999.pth ...


In [2]:
image_path = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data/Data/SWMI/images/177.png"
image_path = r"C:\Users\wouds\Documents\GitHub\Flowsheet_Digitization\data\ASPEN_1\images\images\Aspen PFD_p11-1766.png"
result = get_prediction(detection_model=model, image=image_path)
result = get_sliced_prediction(detection_model=model, image=image_path, slice_height=512, slice_width=512, overlap_height_ratio=0.5, overlap_width_ratio=0.5)
result.export_visuals(export_dir="./output/sahi")
print(result)
print(result.to_coco_annotations())


c:\Users\wouds\Documents\GitHub\Flowsheet_Digitization\.venv\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Performing prediction on 6 number of slices.
[{'image_id': None, 'bbox': [484.3236083984375, 71.40838623046875, 34.41546630859375, 33.12224578857422], 'score': 0.9891179800033569, 'category_id': 136, 'category_name': 'DiscreteField', 'segmentation': [], 'iscrowd': 0, 'area': 1139}, {'image_id': None, 'bbox': [424.9217224121094, 374.15155029296875, 31.876876831054688, 33.676177978515625], 'score': 0.9870033860206604, 'category_id': 136, 'category_name': 'DiscreteField', 'segmentation': [], 'iscrowd': 0, 'area': 1073}, {'image_id': None, 'bbox': [400.84288024902344, 423.3112487792969, 32.528717041015625, 33.4263916015625], 'score': 0.9718191623687744, 'category_id': 136, 'category_name': 'DiscreteField', 'segmentation': [], 'iscrowd': 0, 'area': 1087}, {'image_id': None, 'bbox': [229.5828094482422, 230.23452758789062, 33.32002258300781, 32.84849548339844], 'score': 0.9561728239059448, 'category_id': 136, 'category_name': 'DiscreteField', 'segmentation': [], 'iscrowd': 0, 'area': 1094}, {

In [3]:
result = predict(
    #detection_model=model,
    model_type='detectron2',
    model_path=model_path,
    model_config_path=config_path,
    model_confidence_threshold=0.5,
    model_device="cuda", # or 'cuda:0'
    source=image_path,
    model_category_mapping=category_mapping,
    no_standard_prediction = False,
    no_sliced_prediction = False,
    novisual = False,
    verbose = 1,
    return_dict = True,
    project="output/sahi",
    name="doesthiswork",
    overlap_height_ratio = 0.3,
    overlap_width_ratio = 0.3,
    postprocess_match_threshold=0.5,
    postprocess_class_agnostic=False,
    visual_bbox_thickness = 1,
    visual_text_size = 0.4,
    visual_text_thickness = 1,
    visual_hide_labels = False,
    visual_hide_conf = False,
    visual_export_format = "png",
    export_pickle = True
)

print(result)

[12/12 15:54:02 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Experiments\11. Exp_SWMI_baseline_seg_512_val_seg\results\output\model_0074999.pth ...


12/12/2023 15:54:02 - INFO - fvcore.common.checkpoint -   [Checkpointer] Loading from w:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Experiments\11. Exp_SWMI_baseline_seg_512_val_seg\results\output\model_0074999.pth ...
Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Performing prediction on 4 number of slices.


Performing inference on images: 100%|██████████| 1/1 [00:05<00:00,  5.19s/it]

Prediction time is: 5170.31 ms
Prediction results are successfully exported to output\sahi\doesthiswork13
{'export_dir': WindowsPath('output/sahi/doesthiswork13')}


In [4]:
#image_path = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data/Data/SWMI/images/"
#json_path = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Data\SWMI\SWMI_baseline\test.json"

image_path = r"C:\Users\wouds\Documents\GitHub\Flowsheet_Digitization\data\ASPEN_1\images"
json_path = r"C:\Users\wouds\Documents\GitHub\Flowsheet_Digitization\data\ASPEN_1\test.json"


result = predict(
    #detection_model=model,
    model_type='detectron2',
    model_path=model_path,
    model_config_path=config_path,
    model_confidence_threshold=0.5,
    model_device="cuda", # or 'cuda:0'
    source=image_path,
    model_category_mapping=category_mapping,
    no_standard_prediction = False,
    no_sliced_prediction = True,
    novisual = False,
    verbose = 1,
    return_dict = True,
    project="output/sahi",
    name="result_model_seg_noslice",
    overlap_height_ratio = 0.1,
    overlap_width_ratio = 0.1,
    postprocess_match_threshold=0.5,
    postprocess_class_agnostic=False,
    visual_bbox_thickness = 1,
    visual_text_size = 0.4,
    visual_text_thickness = 1,
    visual_hide_labels = False,
    visual_hide_conf = False,
    visual_export_format = "png",
    dataset_json_path = json_path,
)
"""
result2 = predict(
    #detection_model=model,
    model_type='detectron2',
    model_path=model_path,
    model_config_path=config_path,
    model_confidence_threshold=0.5,
    model_device="cuda", # or 'cuda:0'
    source=image_path,
    model_category_mapping=category_mapping,
    no_standard_prediction = False,
    no_sliced_prediction = False,
    novisual = False,
    verbose = 1,
    return_dict = True,
    slice_height = 512,
    slice_width = 512,
    overlap_height_ratio = 0.1,
    overlap_width_ratio = 0.1,
    postprocess_match_threshold=0.5,
    postprocess_class_agnostic=False,
    project="output/sahi",
    name="result_model_seg_slice_512",
    visual_bbox_thickness = 1,
    visual_text_size = 0.4,
    visual_text_thickness = 1,
    visual_hide_labels = False,
    visual_hide_conf = False,
    visual_export_format = "png",
    dataset_json_path = json_path,
)
"""

indexing coco dataset annotations...


Loading coco annotations: 100%|██████████| 10/10 [00:00<00:00, 260.60it/s]


[12/12 15:54:15 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Experiments\11. Exp_SWMI_baseline_seg_512_val_seg\results\output\model_0074999.pth ...


12/12/2023 15:54:15 - INFO - fvcore.common.checkpoint -   [Checkpointer] Loading from w:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Experiments\11. Exp_SWMI_baseline_seg_512_val_seg\results\output\model_0074999.pth ...
Performing inference on images:  10%|█         | 1/10 [00:01<00:16,  1.84s/it]

Prediction time is: 1810.03 ms


...

Performing inference on images:  90%|█████████ | 9/10 [00:12<00:01,  1.31s/it]

Prediction time is: 1389.85 ms


Performing inference on images: 100%|██████████| 10/10 [00:13<00:00,  1.39s/it]

Prediction time is: 1621.16 ms
Prediction results are successfully exported to output\sahi\result_model_seg_noslice


'\nresult2 = predict(\n    #detection_model=model,\n    model_type=\'detectron2\',\n    model_path=model_path,\n    model_config_path=config_path,\n    model_confidence_threshold=0.5,\n    model_device="cuda", # or \'cuda:0\'\n    source=image_path,\n    model_category_mapping=category_mapping,\n    no_standard_prediction = False,\n    no_sliced_prediction = False,\n    novisual = False,\n    verbose = 1,\n    return_dict = True,\n    slice_height = 512,\n    slice_width = 512,\n    overlap_height_ratio = 0.1,\n    overlap_width_ratio = 0.1,\n    postprocess_match_threshold=0.5,\n    postprocess_class_agnostic=False,\n    project="output/sahi",\n    name="result_model_seg_slice_512",\n    visual_bbox_thickness = 1,\n    visual_text_size = 0.4,\n    visual_text_thickness = 1,\n    visual_hide_labels = False,\n    visual_hide_conf = False,\n    visual_export_format = "png",\n    dataset_json_path = json_path,\n)\n'

In [6]:
#json_path = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Data\SWMI\SWMI_baseline\test.json"
#results_json_path = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\SAHI\output\sahi\result_noslice\result.json"
#output_directory = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\SAHI\output\sahi\result_noslice\eval"
#output_directory_analyse = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\SAHI\output\sahi\result_noslice\analyse"

json_path = r"C:\Users\wouds\Documents\GitHub\Flowsheet_Digitization\data\ASPEN_1\test.json"

#path = r"C:\Users\wouds\Documents\GitHub\Flowsheet_Digitization\output\sahi\result_model_seg_slice_512"
path = r"C:\Users\wouds\Documents\GitHub\Flowsheet_Digitization\output\sahi\result_model_seg_noslice"
results_json_path = os.path.join(path, "result.json")

output_directory = os.path.join(path, "eval")

with open(results_json_path, 'r') as f:
    result = json.load(f)

# Category mapping is incorrect! Add 1 to all values
for i in range(len(result)):
    result[i]['category_id'] += 1

save_path = os.path.join(path, "result_corrected.json")
with open(save_path, 'w') as f:
    json.dump(result, f)

results_json_path = os.path.join(path, "result_corrected.json")

evaluate(dataset_json_path=json_path, result_json_path=results_json_path, out_dir=output_directory, 
type="bbox", classwise=True, max_detections=500, iou_thrs=None, areas=[1024, 9216, 10000000000], return_dict=True)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


DONE (t=0.20s).
Accumulating evaluation results...
DONE (t=0.06s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.167
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.229
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.178
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.315
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.260
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.084
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.221
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.228
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=500 ] = 0.084
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.230
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= lar

{'eval_results': OrderedDict([('bbox_mAP', 0.167),
              ('bbox_mAP50', 0.229),
              ('bbox_mAP75', 0.178),
              ('bbox_mAP_s', 0.221),
              ('bbox_mAP_m', 0.228),
              ('bbox_mAP_l', 0.084),
              ('bbox_mAP50_s', 0.315),
              ('bbox_mAP50_m', 0.26),
              ('bbox_mAP50_l', 0.084),
              ('bbox_mAP_copypaste',
               '0.167 0.178 0.229 0.221 0.228 0.084 0.315 0.260 0.084'),
              ('results_per_category',
               {'bbox_AgitatorDisc_mAP': '0.000',
                'bbox_AgitatorDisc_mAP_s': '-1.000',
                'bbox_AgitatorDisc_mAP_m': '0.000',
                'bbox_AgitatorDisc_mAP_l': '-1.000',
                'bbox_AgitatorDisc_mAP50': '0.000',
                'bbox_AgitatorDisc_mAP50_s': '-1.000',
                'bbox_AgitatorDisc_mAP50_m': '0.000',
                'bbox_AgitatorDisc_mAP50_l': '-1.000',
                'bbox_CheckValveGeneric_mAP': '0.000',
                'bbo

In [7]:
analyse(dataset_json_path=json_path, result_json_path=results_json_path, out_dir=os.path.join(path, "analyse"),
 type="bbox", no_extraplots=True, max_detections=500, areas=[1024, 9216, 10000000000], return_dict=True)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.07s).
Accumulating evaluation results...
DONE (t=0.02s).


In [6]:
json_path = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\Data\SWMI\SWMI_baseline\test.json"
results_json_path = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\SAHI\output\sahi\result_slice_512\result.json"
output_directory = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\SAHI\output\sahi\result_slice_512\eval"
output_directory_analyse = r"W:\staff-umbrella\piresearchStudentsShared\Winston_Oudshoorn\05_Data\SAHI\output\sahi\result_slice_512\analyse"

evaluate(dataset_json_path=json_path, result_json_path=results_json_path, out_dir=output_directory, 
type="bbox", classwise=True, max_detections=500, iou_thrs=None, areas=[1024, 9216, 10000000000], return_dict=True)


loading annotations into memory...
Done (t=0.08s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.48s).
Accumulating evaluation results...
DONE (t=0.24s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.002
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.003
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

{'eval_results': OrderedDict([('bbox_mAP', 0.001),
              ('bbox_mAP50', 0.002),
              ('bbox_mAP75', 0.002),
              ('bbox_mAP_s', 0.001),
              ('bbox_mAP_m', 0.002),
              ('bbox_mAP_l', 0.002),
              ('bbox_mAP50_s', 0.002),
              ('bbox_mAP50_m', 0.002),
              ('bbox_mAP50_l', 0.003),
              ('bbox_mAP_copypaste',
               '0.001 0.002 0.002 0.001 0.002 0.002 0.002 0.002 0.003'),
              ('results_per_category',
               {'bbox_AgitatorDisc_mAP': '0.000',
                'bbox_AgitatorDisc_mAP_s': '-1.000',
                'bbox_AgitatorDisc_mAP_m': '0.000',
                'bbox_AgitatorDisc_mAP_l': '-1.000',
                'bbox_AgitatorDisc_mAP50': '0.000',
                'bbox_AgitatorDisc_mAP50_s': '-1.000',
                'bbox_AgitatorDisc_mAP50_m': '0.000',
                'bbox_AgitatorDisc_mAP50_l': '-1.000',
                'bbox_AgitatorImpeller_mAP': '0.000',
                'bbo

In [22]:
json_path = r"C:\Users\wouds\Documents\GitHub\Flowsheet_Digitization\data\ASPEN_1\test.json"

sahi_folder = r"C:\Users\wouds\Documents\GitHub\Flowsheet_Digitization\output\sahi"

folder_name = "result_noslice"
folder_name = "result_slice_512_postprocess_False_0.5"
folder_name = "result_slice_512_postprocess_True_0.5"
folder_name = "result_slice_1024_postprocess_False_0.5"
folder_name = "result_slice_1024_postprocess_True_0.5"

results_json_path = os.path.join(sahi_folder, folder_name, "result.json")
output_directory = os.path.join(sahi_folder, folder_name, "eval")
output_directory_analyse = os.path.join(sahi_folder, folder_name, "analyse")

with open(results_json_path, 'r') as f:
    result = json.load(f)

# Category mapping is incorrect! Add 1 to all values
for i in range(len(result)):
    result[i]['category_id'] += 1

save_path = os.path.join(sahi_folder, folder_name, "result_corrected.json")
with open(save_path, 'w') as f:
    json.dump(result, f)

results_json_path = os.path.join(sahi_folder, folder_name, "result_corrected.json")

evaluate(dataset_json_path=json_path, result_json_path=results_json_path, out_dir=output_directory, 
type="bbox", classwise=True, max_detections=500, iou_thrs=None, areas=[1024, 9216, 10000000000], return_dict=True)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.20s).
Accumulating evaluation results...
DONE (t=0.06s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.488
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.639
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.574
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.757
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.570
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.500
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.549
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.502
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=500 ] = 0.435
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=50

{'eval_results': OrderedDict([('bbox_mAP', 0.488),
              ('bbox_mAP50', 0.639),
              ('bbox_mAP75', 0.574),
              ('bbox_mAP_s', 0.549),
              ('bbox_mAP_m', 0.502),
              ('bbox_mAP_l', 0.435),
              ('bbox_mAP50_s', 0.757),
              ('bbox_mAP50_m', 0.57),
              ('bbox_mAP50_l', 0.5),
              ('bbox_mAP_copypaste',
               '0.488 0.574 0.639 0.549 0.502 0.435 0.757 0.570 0.500'),
              ('results_per_category',
               {'bbox_AgitatorDisc_mAP': '0.000',
                'bbox_AgitatorDisc_mAP_s': '-1.000',
                'bbox_AgitatorDisc_mAP_m': '0.000',
                'bbox_AgitatorDisc_mAP_l': '-1.000',
                'bbox_AgitatorDisc_mAP50': '0.000',
                'bbox_AgitatorDisc_mAP50_s': '-1.000',
                'bbox_AgitatorDisc_mAP50_m': '0.000',
                'bbox_AgitatorDisc_mAP50_l': '-1.000',
                'bbox_CheckValveGeneric_mAP': '0.750',
                'bbox_